<a href="https://colab.research.google.com/github/chminPark/ml-python/blob/master/VGG%EA%B8%B0%EB%B0%98_%EA%B8%B0%EB%B0%98%EB%B6%84%EB%A5%98_earlyStop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
# Pytorch로 개발하기 위한 Library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

import numpy as np

torch.__version__

'2.1.0+cu118'

In [ ]:
!gdown --id 1L3aXv9A2bn8G9orLqjVWOE_1OY4ycdjs
!mkdir 'chip_data'
!unzip -q chip_data.zip -d chip_data
!rm chip_data.zip

In [87]:
dict_map = {
        'crack, scratch, stamp' : 1,
        'particle' : 2,
        'titled' : 3,
        "와이어끊어짐" : 4,
        "정상": 0
    }
dict_map["와이어끊어짐"]

4

In [88]:
import csv
import os
import pandas as pd
"""
Args:
  directory_string: 이미지가 저장되어 있는 폴더 Path
  output_csv_name: csv 파일 이름
Returns:
  csv file
"""
dict_map = {
        'crack, scratch, stamp' : 1,
        'particle' : 2,
        'titled' : 3,
        "와이어끊어짐" : 4,
        "정상": 0
    }

def build_csv(directory_string, output_csv_name):

    directory = directory_string
    class_list = os.listdir(directory)
    class_list.sort()

    print(class_list)

    with open(output_csv_name, 'w', newline='') as csvfile:

        ### 실습 : CSV 파일 Object 생성
        writer = csv.writer(csvfile, delimiter=',')
        ############################################
        writer.writerow(['file_name', 'file_path', 'class_name', 'class_index']) # CSV의 column 이름을 지정

        ###### 각 folder에 들어가서 각 이미지의 이름을 가져옴
        for class_name in class_list:
          class_path = os.path.join(directory, class_name)
          file_list = os.listdir(class_path) # 해당 파일 내부의 이미지를 확보
          for file_name in file_list:
              file_path = os.path.join(directory, class_name, file_name) #concatenate class folder dir, class name and file name

              writer.writerow([file_name, file_path, class_name, dict_map[class_name]]) #write the file path and class name to the csv file
        #############################

    return

train_folder = os.path.join(os.getcwd(), 'chip_data')
build_csv(train_folder, 'train.csv')
all_df = pd.read_csv('train.csv')


['crack, scratch, stamp', 'particle', 'titled', '와이어끊어짐', '정상']


In [91]:
all_df['class_index'].value_counts()

0    457
1    127
4    101
2     95
3     75
Name: class_index, dtype: int64

In [89]:
all_df

,file_name,file_path,class_name,class_index
0,Sat Nov 25 13-03-11.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
1,Sat Nov 25 15-21-25.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
2,Sat Nov 25 13-04-11.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
3,Sat Nov 25 15-23-04.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
4,Sat Nov 25 13-26-50.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
...,...,...,...,...
850,Sat Nov 25 13-05-41.jpg,/content/chip_data/정상/Sat Nov 25 13-05-41.jpg,정상,0
851,Sat Nov 25 13-44-18.jpg,/content/chip_data/정상/Sat Nov 25 13-44-18.jpg,정상,0
852,Sat Nov 25 13-33-18.jpg,/content/chip_data/정상/Sat Nov 25 13-33-18.jpg,정상,0
853,Sat Nov 25 14-05-47.jpg,/content/chip_data/정상/Sat Nov 25 14-05-47.jpg,정상,0


In [95]:
all_df['class_index'].value_counts()

0    457
1    127
4    101
2     95
3     75
Name: class_index, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_df, test_size=0.2, random_state=42, shuffle=True)

In [94]:
train['class_index'].value_counts(), test['class_index'].value_counts()

(0    369
 1     99
 2     80
 4     79
 3     57
 Name: class_index, dtype: int64,
 0    88
 1    28
 4    22
 3    18
 2    15
 Name: class_index, dtype: int64)

In [96]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_df, test_size=0.2, random_state=42, shuffle=True, stratify=all_df['class_index'])
train['class_index'].value_counts(), test['class_index'].value_counts()

(0    365
 1    102
 4     81
 2     76
 3     60
 Name: class_index, dtype: int64,
 0    92
 1    25
 4    20
 2    19
 3    15
 Name: class_index, dtype: int64)

In [101]:
train.head()

,file_name,file_path,class_name,class_index
377,Sat Nov 25 15-15-14.jpg,/content/chip_data/와이어끊어짐/Sat Nov 25 15-15-14.jpg,와이어끊어짐,4
191,Sat Nov 25 15-38-38.jpg,/content/chip_data/particle/Sat Nov 25 15-38-3...,particle,2
818,Sat Nov 25 14-16-48.jpg,/content/chip_data/정상/Sat Nov 25 14-16-48.jpg,정상,0
41,Sat Nov 25 13-33-27.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
724,Sat Nov 25 14-05-49.jpg,/content/chip_data/정상/Sat Nov 25 14-05-49.jpg,정상,0


In [104]:
# save df to csv
train.to_csv("train.csv", index=0) # dataframe의 index를 저장하지 않기 위해서 index=0을 활용한다.
test.to_csv("test.csv", index=0)

In [105]:
pd.read_csv("train.csv")

,file_name,file_path,class_name,class_index
0,Sat Nov 25 15-15-14.jpg,/content/chip_data/와이어끊어짐/Sat Nov 25 15-15-14.jpg,와이어끊어짐,4
1,Sat Nov 25 15-38-38.jpg,/content/chip_data/particle/Sat Nov 25 15-38-3...,particle,2
2,Sat Nov 25 14-16-48.jpg,/content/chip_data/정상/Sat Nov 25 14-16-48.jpg,정상,0
3,Sat Nov 25 13-33-27.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
4,Sat Nov 25 14-05-49.jpg,/content/chip_data/정상/Sat Nov 25 14-05-49.jpg,정상,0
...,...,...,...,...
679,Sat Nov 25 14-13-03.jpg,/content/chip_data/정상/Sat Nov 25 14-13-03.jpg,정상,0
680,Sat Nov 25 15-03-38.jpg,/content/chip_data/와이어끊어짐/Sat Nov 25 15-03-38.jpg,와이어끊어짐,4
681,Sat Nov 25 15-35-50.jpg,/content/chip_data/particle/Sat Nov 25 15-35-5...,particle,2
682,Sat Nov 25 15-22-50.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1


# Custom데이터를 활용하기 위한 Dataset을 선언
- pytorch의 `dataloader`를 이용하기 위해서는 `torch.utils.data.Dataset` 클래스를 상속한 클래스의 선언이 필요하다.
- 클래스 내에 `__init__`, `__getitem__`, `__len__`의 3개의 메소드를 선언하여 오버라이드한다.

In [107]:
from torch.utils.data import Dataset
Dataset

torch.utils.data.dataset.Dataset

In [106]:
import cv2
import torch
from torch.utils.data import Dataset

class semiconductorDataset(Dataset): # inheritin from Dataset class

    def __init__(self, csv_file, root_dir="", transform=None):
        self.annotation_df = pd.read_csv(csv_file)
        self.root_dir = root_dir # root directory of images, leave "" if using the image path column in the __getitem__ method
        self.transform = transform

    def __len__(self):
        return len(self.annotation_df) # return length (numer of rows) of the dataframe

    ############ 실습 : annotation_df에 있는 이미지를 읽어 들어서, 변경한후
    ############        읽어 들인 값을 return 하는 함수를 작성한다.
    def __getitem__(self, idx):

        image_path = os.path.join(self.root_dir, self.annotation_df.iloc[idx, 1]) #use image path column (index = 1) in csv file
        image = cv2.imread(image_path) # read image by cv2
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert from BGR to RGB for matplotlib

        if self.transform:
            image = self.transform(image)

        #### 이미지에 대한 추가 정보 확인
        class_name = self.annotation_df.iloc[idx, 2] # use class name column (index = 2) in csv file
        class_index = self.annotation_df.iloc[idx, 3] # use class index column (index = 3) in csv file

        return image, class_name, class_index

In [42]:
# https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.Grayscale(), # Gray Scale로 변경
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

transform_test = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.Grayscale(), # Gray Scale로 변경
])

##### 실습 : train_dataset 선언
##### 실습 : test_dataset 선언
dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)
test_dataset = semiconductorDataset(csv_file='test.csv', root_dir="", transform=transform_test)

In [109]:
# dataset을 train 과 validation데이터로 나눈다.
# Pytorch DataSet 을 나눌때에는 random_split을 수행한다.
from torch.utils.data import random_split

train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len

train_dataset, val_dataset = random_split(dataset, [train_len, val_len])
len(train_dataset), len(val_dataset)


(547, 137)

# Batch 단위 학습을 위한 DataLoader 선언
 - Dataset내부의 샘플들을 batch 크기로 추출
 - Batch Size는 1step에 들어간 데이터의 개수
 - Epoch 마다 데이터를 섞어(Shuffle) Overfitting을 방지
 - 병렬처리를 지원하여 데이터 검색 속도를 향상


# VGG6를 이용하여 학습을 수행한다
![](https://drive.google.com/uc?export=view&id=1vLrvhxczx1ZCOH05cxzzESylaOaF1Uj2)


In [117]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

class VGG_BatchNormTorch(nn.Module):
  def __init__(self, in_channels, out_features):
    super(VGG_BatchNormTorch, self).__init__()

    ###### 실습 : Gray이미지 이므로 in_channel =1 로 선언한다
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1)
    self.norm1 = torch.nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
    self.norm2 = torch.nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.norm3 = torch.nn.BatchNorm2d(64)

    self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
    self.norm4 = torch.nn.BatchNorm2d(64)

    self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    #### 실습 : Batch Norm 을 Pytorch의 값으로 변경한다
    self.norm5 = torch.nn.BatchNorm2d(128)
    self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    #### 실습 : Batch Norm 을 Pytorch의 값으로 변경한다
    self.norm6 = torch.nn.BatchNorm2d(128)

    self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2) #Maxpooling layer to change feature size
    self.avg_pool = nn.AdaptiveAvgPool2d(output_size = (1, 1)) #Note that average pooling layer is not adopted in original VGG architecture. We use average pooling layer to make the architecture for experiment simple.

    ###### 실습 : 11개의 Label이 있으므로 out_features=11로 선언한다
    self.fc = nn.Linear(in_features=128, out_features=out_features)

  def forward(self, x):
    #### 실습 : Batch Norm을 Convolution 이후에 선언
    x = self.norm1(self.conv1(x))
    x = F.relu(x)
    x = self.norm2(self.conv2(x))
    x = F.relu(x)
    x = self.max_pool(x)

    x = self.norm3(self.conv3(x))
    x = F.relu(x)
    x = self.norm4(self.conv4(x))
    x = F.relu(x)
    x = self.max_pool(x)

    x = self.norm5(self.conv5(x))
    x = F.relu(x)
    x = self.norm6(self.conv6(x))
    x = F.relu(x)

    x = self.avg_pool(x)
    x = x.view(-1, 128)
    x = self.fc(x)
    return x

In [119]:
class EarlyStopping:

    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [120]:
def train(model, data_loader, val_loader, early_stopping, criterion, optimizer, n_epoch):
  ### 실습 : 학습 중임을 알림
  model.train() ###
  for epoch in range(n_epoch):
    running_loss = 0
    for i, (images, _, labels) in enumerate(data_loader):

      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if (i + 1) % 100 == 0:
        print('iteration: [{}/{}]'.format(i + 1, len(data_loader)))

    # 각 epoch 이후 validation 수행
    model.eval()
    val_loss = 0
    for images, _, labels in val_loader:
      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      vloss = criterion(outputs, labels)

      val_loss += vloss

    # early stopping을 체크
    early_stopping(val_loss, model)

    # overfitting 발생 시 학습 종료
    if early_stopping.early_stop:
      break

    # 원래 상태로 돌아옴
    model.train()
    running_loss = running_loss/len(data_loader)
    val_loss = val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}, train_loss = {running_loss:.3f}, val_loss = {val_loss:.3f}')

In [121]:
def eval(model, data_loader):
  #### 실습 : 평가를 위해서는 eval()을 선언
  model.eval()
  total = 0
  correct = 0

  preds = []
  trues = []
  #### 실습 : Gradient를 타지 않아야 한다
  with torch.no_grad():
    for images, _, labels in data_loader:
      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      preds.append(predicted.detach().cpu().numpy())
      trues.append(labels.detach().cpu().numpy())

    accuracy = 100 * correct / total

  print('Test Accuracy: {}%'.format(accuracy))
  return preds, trues


In [122]:
def reset_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [123]:
#### 실습 : torch.utils.DataLoader를 이용하여 데이터를 load. Batch 크기를 10으로 한다.
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=10, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [124]:
reset_seed(2020)
criterion = nn.CrossEntropyLoss()
vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
optimizer = optim.Adam(params=vgg_batchnorm_model.parameters())

earlystop = EarlyStopping(patience=7)

train(vgg_batchnorm_model, train_dataloader, val_dataloader, earlystop,  criterion, optimizer, n_epoch=50)

Epoch 1, train_loss = 1.372, val_loss = 1.144
Epoch 2, train_loss = 1.295, val_loss = 1.130
EarlyStopping counter: 1 out of 7
Epoch 3, train_loss = 1.271, val_loss = 1.151
EarlyStopping counter: 2 out of 7
Epoch 4, train_loss = 1.244, val_loss = 1.280
Epoch 5, train_loss = 1.201, val_loss = 1.103
Epoch 6, train_loss = 1.137, val_loss = 0.975
Epoch 7, train_loss = 1.058, val_loss = 0.875
EarlyStopping counter: 1 out of 7
Epoch 8, train_loss = 1.004, val_loss = 1.395
EarlyStopping counter: 2 out of 7
Epoch 9, train_loss = 0.931, val_loss = 0.895
Epoch 10, train_loss = 0.848, val_loss = 0.815
Epoch 11, train_loss = 0.767, val_loss = 0.756
Epoch 12, train_loss = 0.740, val_loss = 0.608
EarlyStopping counter: 1 out of 7
Epoch 13, train_loss = 0.660, val_loss = 1.019
EarlyStopping counter: 2 out of 7
Epoch 14, train_loss = 0.589, val_loss = 1.250
EarlyStopping counter: 3 out of 7
Epoch 15, train_loss = 0.557, val_loss = 0.941
EarlyStopping counter: 4 out of 7
Epoch 16, train_loss = 0.569, va

# 정확도를 평가해 봅시다
* 데이터가 너무 적어서 평가 데이터에 대한 정확도를 봅니다

In [125]:
# early stop 된 객체를 load 한다.
state_dict = torch.load('checkpoint.pt')
vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
vgg_batchnorm_model.load_state_dict(state_dict)


<All keys matched successfully>

In [126]:
preds_r, trues_r = eval(vgg_batchnorm_model, train_dataloader)

Test Accuracy: 89.39670932358318%


In [127]:
preds, trues = eval(vgg_batchnorm_model, test_dataloader)

Test Accuracy: 83.04093567251462%


In [128]:
prediction_label = np.stack(preds[:-1]).flatten()
test_labels = np.stack(trues[:-1]).flatten()
prediction_label, test_labels

(array([0, 3, 0, 1, 0, 0, 1, 3, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 4, 1, 0, 1, 2, 2, 4, 0, 1, 0, 0, 0, 0,
        1, 0, 3, 0, 1, 0, 3, 0, 4, 2, 3, 0, 3, 1, 0, 4, 0, 0, 0, 1, 3, 4,
        2, 4, 0, 0, 3, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 1, 4, 2, 0, 0, 1,
        0, 0, 3, 1, 0, 0, 1, 4, 0, 3, 4, 0, 4, 2, 0, 0, 4, 2, 4, 0, 0, 1,
        4, 0, 0, 4, 0, 0, 0, 0, 1, 1, 0, 1, 4, 4, 3, 0, 4, 3, 1, 1, 4, 0,
        3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 3, 0, 4, 3, 0, 4, 0, 1,
        0, 1, 0, 3, 3, 0, 0, 2, 2, 0, 4, 0, 1, 0, 0, 4]),
 array([0, 0, 0, 1, 0, 0, 1, 3, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 4, 1, 0, 0, 2, 2, 4, 1, 1, 0, 0, 1, 0,
        3, 0, 0, 0, 1, 0, 3, 0, 4, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 3, 3, 4,
        2, 4, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 4, 2, 0, 0, 1,
        0, 0, 1, 0, 0, 2, 1, 4, 0, 3, 4, 0, 0, 1, 0, 0, 4, 2, 4, 0, 0, 1,
        0, 1, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 4, 4, 3, 0

In [129]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, prediction_label))

              precision    recall  f1-score   support

           0       0.94      0.82      0.87        92
           1       0.66      0.76      0.70        25
           2       0.89      0.89      0.89        18
           3       0.55      0.73      0.63        15
           4       0.87      1.00      0.93        20

    accuracy                           0.83       170
   macro avg       0.78      0.84      0.80       170
weighted avg       0.85      0.83      0.83       170



# RGB로 학습 후 평가해 본다

In [ ]:
# https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

transform_test = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
])

##### 실습 : train_dataset 선언
##### 실습 : test_dataset 선언
dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)
test_dataset = semiconductorDataset(csv_file='test.csv', root_dir="", transform=transform_test)

In [ ]:
# dataset을 train 과 validation데이터로 나눈다.
# Pytorch DataSet 을 나눌때에는 random_split을 수행한다.
from torch.utils.data import random_split

train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len

train_dataset, val_dataset = random_split(dataset, [train_len, val_len])
len(train_dataset), len(val_dataset)

(547, 137)

In [ ]:
#### 실습 : torch.utils.DataLoader를 이용하여 데이터를 load. Batch 크기를 10으로 한다.
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=10, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [36]:
reset_seed(2020)
criterion = nn.CrossEntropyLoss()
############### 실습 : 입력 Channel의 개수는 3
vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=3, out_features=5).to("cuda")
optimizer = optim.Adam(params=vgg_batchnorm_model.parameters(), lr=0.00005)
earlystop = EarlyStopping(patience=7)

train(vgg_batchnorm_model, train_dataloader, val_dataloader, earlystop,  criterion, optimizer, n_epoch=50)

Epoch 1, train_loss = 1.371, val_loss = 1.292
EarlyStopping counter: 1 out of 7
Epoch 2, train_loss = 1.252, val_loss = 1.424
EarlyStopping counter: 2 out of 7
Epoch 3, train_loss = 1.232, val_loss = 1.322
Epoch 4, train_loss = 1.154, val_loss = 1.167
EarlyStopping counter: 1 out of 7
Epoch 5, train_loss = 1.090, val_loss = 1.365
Epoch 6, train_loss = 1.009, val_loss = 1.013
EarlyStopping counter: 1 out of 7
Epoch 7, train_loss = 0.907, val_loss = 1.418
Epoch 8, train_loss = 0.901, val_loss = 0.998
EarlyStopping counter: 1 out of 7
Epoch 9, train_loss = 0.718, val_loss = 1.011
Epoch 10, train_loss = 0.671, val_loss = 0.798
EarlyStopping counter: 1 out of 7
Epoch 11, train_loss = 0.633, val_loss = 1.396
Epoch 12, train_loss = 0.563, val_loss = 0.718
EarlyStopping counter: 1 out of 7
Epoch 13, train_loss = 0.547, val_loss = 1.562
Epoch 14, train_loss = 0.476, val_loss = 0.634
EarlyStopping counter: 1 out of 7
Epoch 15, train_loss = 0.474, val_loss = 0.762
EarlyStopping counter: 2 out of 

In [37]:
preds_r, trues_r = eval(vgg_batchnorm_model, train_dataloader)

Test Accuracy: 87.56855575868373%


In [38]:
preds, trues = eval(vgg_batchnorm_model, test_dataloader)

Test Accuracy: 84.7953216374269%


In [40]:
prediction_label = np.stack(preds[:-1]).flatten()
test_labels = np.stack(trues[:-1]).flatten()
prediction_label, test_labels

(array([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 0, 2, 0, 4, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 3, 0, 3, 0, 0, 4, 0, 0, 0, 0, 0, 4,
        2, 4, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 4, 2, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 4, 0, 3, 0, 0, 0, 0, 0, 0, 4, 2, 4, 0, 0, 1,
        0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 4, 4, 0, 0, 4, 0, 1, 1, 4, 0,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 4, 0, 0, 0, 0, 4]),
 array([0, 0, 0, 1, 0, 0, 1, 3, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 4, 1, 0, 0, 2, 2, 4, 1, 1, 0, 0, 1, 0,
        3, 0, 0, 0, 1, 0, 3, 0, 4, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 3, 3, 4,
        2, 4, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 4, 2, 0, 0, 1,
        0, 0, 1, 0, 0, 2, 1, 4, 0, 3, 4, 0, 0, 1, 0, 0, 4, 2, 4, 0, 0, 1,
        0, 1, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 4, 4, 3, 0

In [ ]:
########### 실습 : 분류 성능을 평가해 봅니다.
from sklearn.metrics import classification_report
print(classification_report(test_labels, prediction_label))

# Ensemble 수행 - 흑백

In [59]:
# dataset 및 dataloader생성
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.Grayscale(), # Gray Scale로 변경
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

transform_test = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.Grayscale(), # Gray Scale로 변경
])

dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)
test_dataset = semiconductorDataset(csv_file='test.csv', root_dir="", transform=transform_test)

# dataset을 train 과 validation데이터로 나눈다.
from torch.utils.data import random_split
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len])

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=10, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [ ]:
# 학습을 반복적으로 수행
for itr in range(5):
  reset_seed(105 + itr)
  criterion = nn.CrossEntropyLoss()
  vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
  optimizer = optim.Adam(params=vgg_batchnorm_model.parameters(), lr=0.00005)
  earlystop = EarlyStopping(patience=7)

  train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)
  val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=10, shuffle=False)

  train(vgg_batchnorm_model, train_dataloader, val_dataloader, earlystop,  criterion, optimizer, n_epoch=50)

  # 저장된 checkpoint를 다른이름으로 저장
  state_dict = torch.load('checkpoint.pt')
  vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
  vgg_batchnorm_model.load_state_dict(state_dict)
  torch.save(vgg_batchnorm_model.state_dict(), f"ckpt_{itr}.pt")
  print(f"Save ckpt_{itr}.pt")


Epoch 1, train_loss = 1.473, val_loss = 1.354
Epoch 2, train_loss = 1.368, val_loss = 1.224
Epoch 3, train_loss = 1.324, val_loss = 1.192
EarlyStopping counter: 1 out of 7
Epoch 4, train_loss = 1.286, val_loss = 1.229
Epoch 5, train_loss = 1.262, val_loss = 1.170
Epoch 6, train_loss = 1.235, val_loss = 1.088
EarlyStopping counter: 1 out of 7
Epoch 7, train_loss = 1.226, val_loss = 1.106
EarlyStopping counter: 2 out of 7
Epoch 8, train_loss = 1.206, val_loss = 1.120
EarlyStopping counter: 3 out of 7
Epoch 9, train_loss = 1.189, val_loss = 1.110
Epoch 10, train_loss = 1.163, val_loss = 0.962
EarlyStopping counter: 1 out of 7
Epoch 11, train_loss = 1.144, val_loss = 1.128
EarlyStopping counter: 2 out of 7
Epoch 12, train_loss = 1.121, val_loss = 1.012
EarlyStopping counter: 3 out of 7
Epoch 13, train_loss = 1.105, val_loss = 1.036
EarlyStopping counter: 4 out of 7
Epoch 14, train_loss = 1.098, val_loss = 1.036
EarlyStopping counter: 5 out of 7
Epoch 15, train_loss = 1.059, val_loss = 0.99

In [71]:
for itr in range(5):
  state_dict = torch.load(f'cktp_{itr}.pt')
  vgg_batchnorm_model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
  vgg_batchnorm_model.load_state_dict(state_dict)
  preds, trues = eval(vgg_batchnorm_model, test_dataloader)

Test Accuracy: 82.45614035087719%
Test Accuracy: 78.94736842105263%
Test Accuracy: 81.87134502923976%
Test Accuracy: 85.38011695906432%
Test Accuracy: 85.96491228070175%


In [81]:
# test accuracy가 높은 3개를 가지고 와서 Ensemble을 하겠습니다.
# Ensemble방식은 결과를 평균하겠습니다.

def eval_ensemble(model_idxs, data_loader):
  #### 실습 : 평가를 위해서는 eval()을 선언

  total = 0
  correct = 0

  preds = []
  trues = []
  #### 실습 : Gradient를 타지 않아야 한다
  with torch.no_grad():
    for images, _, labels in data_loader:
      images, labels = images.cuda(), labels.cuda()

      outputs = []
      for idx in model_idxs:
        state_dict = torch.load(f'cktp_{itr}.pt')
        model = VGG_BatchNormTorch(in_channels=1, out_features=5).to("cuda")
        model.load_state_dict(state_dict)

        model.eval()
        output = model(images)
        outputs.append(output)
      # outputs = [Tensor(10x5), Tensor(10x5), Tensor(10x5)]
      # Ensemble 결과를 모으고
      outputs = torch.stack(outputs, dim=0) # 3 x 10 x 5

      # 평균취한다.
      outputs = torch.mean(outputs, dim=0) # 10 x 5

      _, predicted = torch.max(outputs, 1)


      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      preds.append(predicted.detach().cpu().numpy())
      trues.append(labels.detach().cpu().numpy())

    accuracy = 100 * correct / total

  print('Test Accuracy: {}%'.format(accuracy))
  return preds, trues

preds, trues = eval_ensemble([0,3,4], test_dataloader)


Test Accuracy: 85.96491228070175%


In [83]:
prediction_label = np.stack(preds[:-1]).flatten()
test_labels = np.stack(trues[:-1]).flatten()
from sklearn.metrics import classification_report
print(classification_report(test_labels, prediction_label))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91        92
           1       0.91      0.80      0.85        25
           2       0.68      0.83      0.75        18
           3       0.67      0.67      0.67        15
           4       1.00      0.75      0.86        20

    accuracy                           0.86       170
   macro avg       0.83      0.80      0.81       170
weighted avg       0.87      0.86      0.86       170

